In [2]:
# If the libraries are not yet installed, they can be installed using conda commands similar to the below
# %conda install pandas
# %conda install requests 

# Something like the following may also work if the above does not
# import sys
# !conda install --yes --prefix {sys.prefix} pandas
# !conda install --yes --prefix {sys.prefix} requests

# To install a specific version, add the version to the install command
# E.g., %conda install pandas=1.3.4

# If all else fails, use pip or follow additional advice such as found at
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

# If your plan to use pip (especially if you are not working within a specified conda environment), 
# the pip commands might look like:
# pip install pandas
# pip install requests

# To install a specific version, add the version to the pip install command
# E.g., pip install pandas==1.3.4

In [3]:
# Load necessary libraries
import requests
import pandas as pd
from io import StringIO
import time
import datetime

In [4]:
def get_portland_vancouver_data(date_list, retry_limit=15, highway_id="6", 
                                resolution="00:15:00", detector_choice=None):
    """Function to get portland-vancouver regional area highway data from the provided api"""
    
    # Set API headers
    headers = {
        'Host': 'new.portal.its.pdx.edu:8080',
        'Referer': 'http://new.portal.its.pdx.edu:8080/downloads/'
    }
    
    # Parameters for the while loop, which limits retries upon failure,
    # and for the counter, which loops through the full date range provided in the function call
    n = len(date_list)-1
    counter = 0
    retries = retry_limit
    num_retries = 0
    
    # Initialize empty data frame
    full_df = pd.DataFrame()
    
    while num_retries < retries:
        while counter < n:
            print(f"Reading for date range {date_list[counter]} to {date_list[counter+1]}")

            # Make API calls within try-except block to catch failures
            try:
                start = time.time()
                
                # Set params for API call
                params = (
                    ('start_date', date_list[counter]),
                    ('end_date', date_list[counter+1]),
                    ('highway_id', [highway_id]),
                    ('resolution', resolution),
                    ('format', 'csv')
                )

                # Make API call
                response = requests.get(
                    'http://new.portal.its.pdx.edu:8080/highways/api/freewaydata/',
                    headers = headers, params = params)

                end = time.time()
                print(f"API Call took {round(end-start, 2)} seconds")
                
                # Check that the call was successful, if not raise a generic exception
                state = response.status_code
                if state != 200:
                    raise Exception(state)
                
                # If the call was successful, get the text body from the result and read in to df
                df = pd.read_csv(StringIO(response.text))
                
                # If a detector is specifed in function call, filter to only that detector in result df
                if detector_choice:    
                    df = df.query("detector_id in @detector_choice")

                # Append to full data frame
                full_df = full_df.append(df)

                # Increment the counter
                counter += 1

                # Reset the retry counter
                num_retries = 0

            except Exception as err:
                # If there was an exception, increment the rety counter and alert the user
                num_retries += 1
                print(f"API call failed. Retrying in 2 seconds. This is retry {num_retries} of {retries}")

            # Sleep 2 seconds between API calls
            time.sleep(2)

            print("-------------------------")
            
        break
        
    # Return dataframe with full results
    return full_df

In [5]:
# Create a date list which breaks up. the year 2019 into 5-day long intervals to use with the API calling function
date_list = ['2018-12-31', 
             '2019-01-05', 
             '2019-01-10',
             '2019-01-15',
             '2019-01-20',
             '2019-01-25',
             '2019-01-31',
             '2019-02-01',
             '2019-02-05',
             '2019-02-10',
             '2019-02-15',
             '2019-02-20',
             '2019-02-28',
             '2019-03-01',
             '2019-03-05',
             '2019-03-10',
             '2019-03-15',
             '2019-03-20',
             '2019-03-25',
             '2019-03-31',
             '2019-04-01',
             '2019-04-05',
             '2019-04-10',
             '2019-04-15',
             '2019-04-20',
             '2019-04-25',
             '2019-04-30',
             '2019-05-01',
             '2019-05-05',
             '2019-05-10',
             '2019-05-15',
             '2019-05-20',
             '2019-05-25',
             '2019-05-31',
             '2019-06-01',
             '2019-06-05',
             '2019-06-10',
             '2019-06-15',
             '2019-06-20',
             '2019-06-25',
             '2019-06-30',
             '2019-07-01',
             '2019-07-05',
             '2019-07-10',
             '2019-07-15',
             '2019-07-20',
             '2019-07-25',
             '2019-07-31',
             '2019-08-01',
             '2019-08-05',
             '2019-08-10',
             '2019-08-15',
             '2019-08-20',
             '2019-08-25',
             '2019-08-31',
             '2019-09-01',
             '2019-09-05',
             '2019-09-10',
             '2019-09-15',
             '2019-09-20',
             '2019-09-25',
             '2019-09-30',
             '2019-10-01',
             '2019-10-05',
             '2019-10-10',
             '2019-10-15',
             '2019-10-20',
             '2019-10-25',
             '2019-10-31',
             '2019-11-01',
             '2019-11-05',
             '2019-11-10',
             '2019-11-15',
             '2019-11-20',
             '2019-11-25',
             '2019-11-30',
             '2019-12-01',
             '2019-12-05',
             '2019-12-10',
             '2019-12-15',
             '2019-12-20',
             '2019-12-25',
             '2020-01-01'
]

In [4]:
# Make the API calls for highway 1
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="1", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway1_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 12.11 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 9.77 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 9.99 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 44.91 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 4.8 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 11.74 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 2.9 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 8.12 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 9.21 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 9.24 seconds
---------------

API Call took 13.36 seconds
-------------------------


In [5]:
# Make the API calls for highway 2 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="2", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway2_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 10.12 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 10.1 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 10.28 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 7.69 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 4.3 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 10.83 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 3.01 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 7.21 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 9.07 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 8.94 seconds
--------------

API Call took 13.68 seconds
-------------------------


In [6]:
# Make the API calls for highway 3 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="3", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway3_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 9.01 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 8.18 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 8.32 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 6.81 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 3.64 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 44.84 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 2.12 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 6.55 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 7.76 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 7.6 seconds
----------------

API Call took 13.45 seconds
-------------------------


In [7]:
# Make the API calls for highway 4 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="4", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway4_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 8.2 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 7.06 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 6.97 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 6.41 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 3.54 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 8.51 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 2.33 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 5.85 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 7.0 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 6.75 seconds
------------------

API Call took 13.38 seconds
-------------------------


In [8]:
# Make the API calls for highway 5 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="5", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway5_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 2.48 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 2.32 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 2.37 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 1.91 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 1.25 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 2.74 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 0.95 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 2.21 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 2.51 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 2.34 seconds
----------------

API Call took 3.12 seconds
-------------------------


In [9]:
# Make the API calls for highway 6 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="6", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway6_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 39.21 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 3.04 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 3.23 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 2.52 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 1.94 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 3.43 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 1.08 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 2.75 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 2.95 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 3.18 seconds
---------------

API Call took 3.8 seconds
-------------------------


In [10]:
# Make the API calls for highway 7 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="7", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway7_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 7.54 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 6.98 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 6.97 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 41.66 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 3.15 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 8.11 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 2.25 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 5.95 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 6.87 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 6.73 seconds
---------------

API Call took 9.39 seconds
-------------------------


In [11]:
# Make the API calls for highway 8 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="8", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway8_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 7.41 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 6.42 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 7.09 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 5.5 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 39.08 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 7.81 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 2.36 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 5.59 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 6.31 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 41.93 seconds
---------------

API Call took 9.31 seconds
-------------------------


In [12]:
# Make the API calls for highway 9 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="9", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway9_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 4.34 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 4.24 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 3.93 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 3.24 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 2.36 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 4.51 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 1.65 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 3.47 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 40.13 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 3.86 seconds
---------------

API Call took 4.98 seconds
-------------------------


In [13]:
# Make the API calls for highway 10 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="10", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway10_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 5.06 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 4.75 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 5.16 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 4.27 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 2.7 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 5.51 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 1.62 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 4.35 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 4.67 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 41.53 seconds
----------------

API Call took 6.16 seconds
-------------------------


In [14]:
# Make the API calls for highway 11 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="11", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway11_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 6.87 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 6.76 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 6.38 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 5.87 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 3.48 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 42.67 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 1.88 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 5.91 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 6.84 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 6.87 seconds
---------------

API Call took 8.16 seconds
-------------------------


In [15]:
# Make the API calls for highway 12 and save the data
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="12", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway12_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 5.23 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 5.68 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 5.13 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 4.72 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 2.88 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 42.13 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 1.88 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 4.72 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 5.6 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 5.46 seconds
----------------

API Call took 7.51 seconds
-------------------------


In [6]:
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="50", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway50_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 5.43 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 5.2 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 5.07 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 5.14 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 4.82 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 60.46 seconds
API call failed. Retrying in 2 seconds. This is retry 1 of 15
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 6.08 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 1.8 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 3.95 seconds
-------------------------
Reading for date range 2019-02

API Call took 5.58 seconds
-------------------------
Reading for date range 2019-11-10 to 2019-11-15
API Call took 5.14 seconds
-------------------------
Reading for date range 2019-11-15 to 2019-11-20
API Call took 4.77 seconds
-------------------------
Reading for date range 2019-11-20 to 2019-11-25
API Call took 4.98 seconds
-------------------------
Reading for date range 2019-11-25 to 2019-11-30
API Call took 4.97 seconds
-------------------------
Reading for date range 2019-11-30 to 2019-12-01
API Call took 1.62 seconds
-------------------------
Reading for date range 2019-12-01 to 2019-12-05
API Call took 4.29 seconds
-------------------------
Reading for date range 2019-12-05 to 2019-12-10
API Call took 4.81 seconds
-------------------------
Reading for date range 2019-12-10 to 2019-12-15
API Call took 4.99 seconds
-------------------------
Reading for date range 2019-12-15 to 2019-12-20
API Call took 5.07 seconds
-------------------------
Reading for date range 2019-12-20 to 2

In [17]:
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="51", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway51_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 5.48 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 5.03 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 4.57 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 4.83 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 5.11 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 41.35 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 1.97 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 4.3 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 5.13 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 4.79 seconds
----------------

API Call took 6.9 seconds
-------------------------


In [18]:
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="52", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway52_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 2.32 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 1.75 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 2.01 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 2.12 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 2.05 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 2.06 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 0.84 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 37.5 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 2.06 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 1.73 seconds
----------------

API Call took 3.1 seconds
-------------------------


In [19]:
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="53", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway53_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 2.32 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 2.14 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 2.36 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 2.08 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 2.12 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 2.28 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 36.32 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 1.68 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 2.13 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 1.88 seconds
---------------

API Call took 3.25 seconds
-------------------------


In [20]:
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="618", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway618_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 1.71 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 1.45 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 1.42 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 35.45 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 1.26 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 1.53 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 0.77 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 1.53 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 1.72 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 1.71 seconds
---------------

API Call took 1.92 seconds
-------------------------


In [21]:
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="620", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway620_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 1.39 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 1.55 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 1.52 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 1.16 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 36.99 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 1.69 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 0.74 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 1.62 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 35.75 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 1.8 seconds
---------------

API Call took 36.44 seconds
-------------------------


In [22]:
df = get_portland_vancouver_data(date_list, retry_limit=15, highway_id="621", 
                                 resolution="00:15:00")
df.to_csv('Data/Raw/Portland/all_sensors_highway621_portland.csv', index=False)

Reading for date range 2018-12-31 to 2019-01-05
API Call took 1.61 seconds
-------------------------
Reading for date range 2019-01-05 to 2019-01-10
API Call took 1.46 seconds
-------------------------
Reading for date range 2019-01-10 to 2019-01-15
API Call took 1.51 seconds
-------------------------
Reading for date range 2019-01-15 to 2019-01-20
API Call took 34.94 seconds
-------------------------
Reading for date range 2019-01-20 to 2019-01-25
API Call took 1.44 seconds
-------------------------
Reading for date range 2019-01-25 to 2019-01-31
API Call took 1.66 seconds
-------------------------
Reading for date range 2019-01-31 to 2019-02-01
API Call took 0.71 seconds
-------------------------
Reading for date range 2019-02-01 to 2019-02-05
API Call took 1.31 seconds
-------------------------
Reading for date range 2019-02-05 to 2019-02-10
API Call took 1.61 seconds
-------------------------
Reading for date range 2019-02-10 to 2019-02-15
API Call took 1.51 seconds
---------------

API Call took 2.17 seconds
-------------------------
